In [2]:
import os

In [3]:
os.chdir('/content/drive/MyDrive/컵퓨터비전 시즌4/6. 프로젝트/6. Final Project/data')

In [4]:
!pwd

/content/drive/MyDrive/컵퓨터비전 시즌4/6. 프로젝트/6. Final Project/data


In [5]:
# yes : 명령어 실행 시 모든 질문에 자동으로 "yes" 라고 응답
# ffmpeg : 비디오와 오디오 파일을 처리하는 요픈 소스 툴
# -ss 00:00:00 : 비디오의 시작 지점 설정. 00:00:00 시작부터 작업을 수행
# -i MOT17-13.mp4 : 입력 파일 지정
# -t 00:00:10 -vf : 비디오에서 10초간의 구간을 추출
# -vf fps=30 : 비디오 필터로 초당 30프레임으로 조정
# MOT17-13_10.mp4 : 결과로 나올 새로운 비디오 파일 이름

!yes | ffmpeg -ss 00:00:00 -i MOT17-13.mp4 -t 00:00:10 -vf fps=30 MOT17-13_10.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [6]:
!pip install boxmot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━

In [7]:
import torch
import torchvision
import cv2
import time
import numpy as np
from pathlib import Path
from google.colab.patches import cv2_imshow
from torchvision import transforms
from boxmot import BotSort
from IPython.display import HTML
from base64 import b64encode
from tqdm import tqdm
from moviepy.editor import *

/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
  if event.key is 'enter':



In [8]:
data_root = '/content/drive/MyDrive/컵퓨터비전 시즌4/6. 프로젝트/6. Final Project/data'
vid_root = f'{data_root}/MOT17-13_10.mp4'

In [9]:
device = torch.device(0) if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [10]:
tracker = BotSort(
    reid_weights=Path('osnet_x0_25_msmt17.pt'),
    device=device,
    half=False
)

2025-02-10 09:28:33.392 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v11.0.7 🚀 Python-3.11.11 torch-2.2.2+cu121CPU
2025-02-10 09:28:34.246 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:183 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


In [11]:
model = dict()

model['det'] = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model['det'].eval().to(device)
model['pose'] = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True)
model['pose'].eval().to(device)
model['seg'] = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model['seg'].eval().to(device)

  warnings.warn(

  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 80.9MB/s]
  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/keypointrcnn_resnet50_fpn_coco-fc266e95.pth" to /root/.cache/torch/hub/checkpoints/keypointrcnn_resnet50_fpn_coco-fc266e95.pth
100%|██████████| 226M/226M [00:02<00:00, 110MB/s]
  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 102MB/s]


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [12]:
def get_color(track_id):
  np.random.seed(int(track_id))
  return tuple(np.random.randint(0, 255, 3). tolist())

In [13]:
# example
video_file = open(vid_root, "rb").read()

video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
HTML(f"""<video width=400px controls><source src="{video_url}"></video>""")

Output hidden; open in https://colab.research.google.com to view.

In [14]:
def boxmot_run(vid_root, dst_name, device, tracking_type, model, out_width=400):
  vid = cv2.VideoCapture(vid_root)

  w = round(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
  h = round(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = vid.get(cv2.CAP_PROP_FPS)

  fourcc = cv2.VideoWriter_fourcc(*'DIVX')
  dst = cv2.VideoWriter(f'{data_root}/{dst_name}', fourcc, fps, (w, h))
  if tracking_type =='det':
    boxmot_det(vid, dst, dst_name, device, model['det'])
  elif tracking_type =='pose':
    boxmot_pose(vid, dst, dst_name, device, model['pose'])
  elif tracking_type =='seg':
    boxmot_seg(vid, dst, dst_name, device, model['seg'])
  else:
    raise NameError('invalid Type')

In [15]:
def boxmot_det(vid, dst, dst_name, device, model):
  frame_cnt = round(vid.get(cv2.CAP_PROP_FRAME_COUNT))
  with tqdm(total=frame_cnt, desc = 'video tracking: ') as pbar:
    while True:
      ret, frame = vid.read()

      if not ret:
        break

      frame_tensor = transforms.functional.to_tensor(frame).to(device)

      with torch.no_grad():
        detections = model([frame_tensor])[0]

      confidence_threshold = 0.5
      dets = []
      for i, score in enumerate(detections['scores']):
        if score >= confidence_threshold:
          bbox = detections['boxes'][i].cpu().numpy()
          label = detections['labels'][i].item()
          conf = score.item()
          dets.append([*bbox, conf, label])

      dets = np.array(dets)
      res = tracker.update(dets, frame)

      # # for test
      # print(dets)
      # print(bbox)
      # break

      tracker.plot_results(frame, show_trajectories=True)
      dst.write(frame)
      pbar.update(1)


  vid.release()

In [16]:
def boxmot_pose(vid, dst, dst_name, device, model):
  frame_cnt = round(vid.get(cv2.CAP_PROP_FRAME_COUNT))
  with tqdm(total=frame_cnt, desc = 'video tracking: ') as pbar:
    while True:
      ret, frame = vid.read()

      if not ret:
        break

      frame_tensor = transforms.functional.to_tensor(frame).unsqueeze(0).to(device)

      with torch.no_grad():
        detections = model(frame_tensor)[0]

      confidence_threshold = 0.5
      dets = []
      keypoints = []
      for i, score in enumerate(detections['scores']):
        if score >= confidence_threshold:
          x1, y1, x2, y2 = detections['boxes'][i].cpu().numpy()
          cls = detections['labels'][i].item()
          conf = score.item()
          dets.append([x1, y1, x2, y2, conf, cls])

          keypoint = detections['keypoints'][i].cpu().numpy().tolist()
          keypoints.append(keypoint)

      dets = np.array(dets)
      tracks = tracker.update(dets, frame)

      # # for test
      # print(dets)
      # print(x1, y1, x2, y2)
      # break

      if len(tracks)>0:
        inds = tracks[:, 7].astype('int')

        keypoints = [keypoints[i] for i in inds if i < len(keypoints)]

        for i, track in enumerate(tracks):
          x1, y1, x2, y2, track_id, conf, cls  = track[:7].astype('int')
          color = get_color(track_id)

          cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

          cv2.putText(frame, f'ID: {track_id}, Conf: {conf:2f}, Class: {cls}',
                      (x1,y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

          if i < len(keypoints):
            kp = keypoints[i]
            for point in kp:
              x, y, confidence = int(point[0]), int(point[1]), point[2]
              if confidence > 0.5:
                cv2.circle(frame, (x, y), 3, color, -1)

      dst.write(frame)
      pbar.update(1)

    vid.release()


In [17]:
def boxmot_seg(vid, dst, dst_name, device, model):
  frame_cnt = round(vid.get(cv2.CAP_PROP_FRAME_COUNT))
  with tqdm(total=frame_cnt, desc = 'video tracking: ') as pbar:
    while True:
      ret, frame = vid.read()

      if not ret:
        break

      frame_tensor = transforms.functional.to_tensor(frame).unsqueeze(0).to(device)

      with torch.no_grad():
        detections = model(frame_tensor)[0]

      confidence_threshold = 0.5
      dets = []
      masks = []
      for i, score in enumerate(detections['scores']):
        if score >= confidence_threshold:
          x1, y1, x2, y2 = detections['boxes'][i].cpu().numpy()
          cls = detections['labels'][i].item()
          conf = score.item()
          dets.append([x1, y1, x2, y2, conf, cls])

          mask = detections['masks'][i,0].cpu().numpy()
          masks.append(mask)


      dets = np.array(dets)
      tracks = tracker.update(dets, frame)

      if len(tracks)>0:
        inds = tracks[:, 7].astype(int)
        if len(masks)>0:
          masks = [masks[i] for i in inds if i < len(masks)]
        for track, mask in zip(tracks, masks):
          track_id = int(track[4])
          color = get_color(track_id)

          if mask is not None:
            mask = (mask > 0.5).astype(np.uint8)

            frame[mask ==1] = frame[mask ==1] * 0.5 + np.array(color) * 0.5

          x1, y1, x2, y2 = track[:4].astype('int')
          cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

          conf = track[5]
          cls = track[6]
          cv2.putText(frame, f'ID: {track_id}, Conf: {conf:2f}, Class: {cls}',
                      (x1,y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


      dst.write(frame)
      pbar.update(1)

    vid.release()


In [18]:
boxmot_run(vid_root = vid_root, dst_name = 'vid_det.avi', device=device, tracking_type='det', model=model)

video tracking: 100%|██████████| 300/300 [58:51<00:00, 11.77s/it]


In [27]:
boxmot_run(vid_root = vid_root, dst_name = 'vid_pose.avi', device=device, tracking_type='pose', model=model)

video tracking: 100%|██████████| 300/300 [1:34:13<00:00, 18.84s/it]


In [18]:
boxmot_run(vid_root = vid_root, dst_name = 'vid_seg.avi', device=device, tracking_type='seg', model=model)

video tracking: 100%|██████████| 300/300 [1:14:37<00:00, 14.93s/it]
